In [1]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [2]:
from jax import grad, random
import jax.numpy as np
from neural_tangents import stax
import neural_tangents as nt

In [3]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(512), stax.Relu(),
    stax.Dense(512), stax.Relu(),
    stax.Dense(1)
)

key1, key2 = random.split(random.PRNGKey(1), 2)
x1 = random.normal(key1, (10, 100)) # Train
x2 = random.normal(key2, (20, 100)) # Test

kernel = kernel_fn(x1, x2, 'nngp')

RuntimeError: CUDA operation failed: cudaGetErrorString symbol not found.

In [4]:
kernel.shape

NameError: name 'kernel' is not defined

In [5]:
# Get kernel of a single type
nngp = kernel_fn(x1, x2, 'nngp') # (10, 20) np.ndarray
ntk = kernel_fn(x1, x2, 'ntk') # (10, 20) np.ndarray

# Get kernels as a namedtuple
both = kernel_fn(x1, x2, ('nngp', 'ntk'))
both.nngp == nngp  # True
both.ntk == ntk  # True

# Unpack the kernels namedtuple
nngp, ntk = kernel_fn(x1, x2, ('nngp', 'ntk'))

NameError: name 'x1' is not defined

In [18]:
import neural_tangents as nt

x_train, x_test = x1, x2
y_train = random.uniform(key1, shape=(10, 1))  # training targets

predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, x_train,
                                                      y_train)

y_test_nngp = predict_fn(x_test=x_test, get='nngp')
# (20, 1) np.ndarray test predictions of an infinite Bayesian network

y_test_ntk = predict_fn(x_test=x_test, get='ntk')
# (20, 1) np.ndarray test predictions of an infinite continuous
# gradient descent trained network at convergence (t = inf)

# Get predictions as a namedtuple
both = predict_fn(x_test=x_test, get=('nngp', 'ntk'))
both.nngp == y_test_nngp  # True
both.ntk == y_test_ntk  # True

# Unpack the predictions namedtuple
y_test_nngp, y_test_ntk = predict_fn(x_test=x_test, get=('nngp', 'ntk'))

In [19]:
!nvidia-smi

Fri Oct 30 09:40:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   36C    P8    18W / 250W |  10185MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 23%   28C    P8     9W / 250W |  10141MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [10]:
_, _, kernel_fn = stax.serial(stax.Dense(1), stax.Relu(), stax.Dense(1))

In [12]:
kernel_fn_batched = nt.batch(kernel_fn, batch_size=5)
x1, x2 = np.ones((40, 10)), np.ones((80, 10))

In [15]:
kernel_fn_batched(x1, x2, ('nngp'))

/home/assistant/jax/jax/_src/lax/lax.py:6135: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))


DeviceArray([[0.50000006, 0.50000006, 0.50000006, ..., 0.50000006,
              0.50000006, 0.50000006],
             [0.50000006, 0.50000006, 0.50000006, ..., 0.50000006,
              0.50000006, 0.50000006],
             [0.50000006, 0.50000006, 0.50000006, ..., 0.50000006,
              0.50000006, 0.50000006],
             ...,
             [0.50000006, 0.50000006, 0.50000006, ..., 0.50000006,
              0.50000006, 0.50000006],
             [0.50000006, 0.50000006, 0.50000006, ..., 0.50000006,
              0.50000006, 0.50000006],
             [0.50000006, 0.50000006, 0.50000006, ..., 0.50000006,
              0.50000006, 0.50000006]], dtype=float32)